In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from itertools import *
import seaborn as sns
from matplotlib.pyplot import figure

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
#from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
import matplotlib.pyplot as plt

#from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.feature_selection import mutual_info_regression

from sklearn.datasets import make_classification, make_regression
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import mutual_info_regression
from scipy import stats
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

## Датасеты и обработка

In [9]:

pd.set_option('future.no_silent_downcasting', True)

student = pd.read_csv("/kaggle/input/student-habits-vs-academic-performance/student_habits_performance.csv")
student = student.replace({'No':1, 'Yes':2, 'Female':1, 'Male':2,'Other':3})
student['diet_quality'] = student['diet_quality'].replace({'Fair':2, 'Good':3, 'Poor':1})
student['parental_education_level'] = student['parental_education_level'].replace({'Master':3, 'High School':1, 'Bachelor':2})
student['internet_quality'] = student['internet_quality'].replace({'Average':2, 'Poor':1, 'Good':3})
student = student.dropna()
student[['gender', 'part_time_job',
         'diet_quality',
         'parental_education_level',
         'internet_quality',
         'extracurricular_participation']] = student[['gender',
                                                      'part_time_job', 
                                                      'diet_quality',
                                                      'parental_education_level',                   
                                                      'internet_quality',
                                                      'extracurricular_participation']].astype('float32')
student = student.drop(columns='student_id')

person = pd.read_csv("/kaggle/input/extrovert-vs-introvert-behavior-data/personality_datasert.csv")
person = person.replace({'No':1, 'Yes':2, 'Extrovert':1, 'Introvert':0})
person[['Stage_fear','Drained_after_socializing',
'Personality']] = person[['Stage_fear','Drained_after_socializing',
'Personality']].astype('int32')


heart2 = pd.read_csv("/kaggle/input/cardiovascular-disease-dataset/cardio_train.csv", delimiter=';')
heart2 = heart2.drop(columns='id')
heart2['smoke'] = heart2['smoke'].replace({0:1, 1:2})
heart2 ['alco'] = heart2 ['alco'] .replace({0:1, 1:2})
heart2['active'] = heart2['active'].replace({0:1, 1:2})

#heart2 = heart2.replace({'Master':3, 'High School':1, 'Bachelor':2
                        
iris = pd.read_csv('/kaggle/input/iris/Iris.csv')
iris = iris.drop(columns='Id')
le = LabelEncoder()
# Fit and transform the 'Size' column
iris['Species'] = le.fit_transform(iris['Species'])

creditfraud = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')
onlinefraud = pd.read_csv('/kaggle/input/online-payment-fraud-detection/onlinefraud.csv')
onlinefraud = onlinefraud.drop(columns=['nameOrig','nameDest','isFlaggedFraud']) 
onlinefraud['type'] = onlinefraud['type'].replace({'PAYMENT':1,
                                                  'TRANSFER':2,
                                                  'CASH_OUT':3, 
                                                  'DEBIT':4, 
                                                  'CASH_IN':4})
onlinefraud['type'] = onlinefraud['type'].astype('int32')

bankruptcy = pd.read_csv('/kaggle/input/company-bankruptcy-prediction/data.csv')
b = bankruptcy['Bankrupt?']
bankruptcy = bankruptcy.drop(columns=['Bankrupt?']) 
bankruptcy['Bankrupt?'] = b

In [32]:

class Generator():

    def __init__(self, X, y, n_columns = 2):
       self.X = X
       self.y = y
       self.n_columns = n_columns # hiperparametr for generators witch split range of combination


    def split_list_by_chunks(self, data, n_columns):
        for i in range(0, len(data), self.n_columns ):
            yield data[i:i + self.n_columns]

    def multiply_gen(self, by=2, column_list=None): ## return np with annotation 
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in combinations(name_columns, by):
            combinations_list.append(name_col)
        
        combinations_list = self.split_list_by_chunks(combinations_list, self.n_columns )
        for seq in combinations_list:
            B = np.empty((len(seq),self.X.shape[0]))
            list_of_name = []
            for i, tup in enumerate(seq):
                A = np.ones(self.X.shape[0])
                list_of_name.append(f'multiply of - {" ".join(str(item) for item in tup)}')
                for col in tup:
                    A *= self.X[col].to_numpy()
                B[i] = A
            yield B, list_of_name
 

    def multiply(self, by=2, column_list=None, with_all_data=True):  ## return dataframe 
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in combinations(name_columns, by):
            combinations_list.append(name_col)      
        B = np.empty((len(combinations_list),self.X.shape[0]))
        list_of_name = []
        for i, tup in enumerate(combinations_list):
            A = np.ones(self.X.shape[0])
            list_of_name.append(f'multiply of - {" ".join(str(item) for item in tup)}')
            for col in tup:
                A *= self.X[col]
            B[i] = A
        df = pd.DataFrame(B.T, columns=list_of_name)
        if with_all_data:
            return pd.concat([df, self.X, self.y], axis=1)   
        else:     
            return df
        

    def devision_on_last_gen(self, by=3, column_list=None):
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in permutations(name_columns, by):
            combinations_list.append(name_col)
        combinations_list = self.split_list_by_chunks(combinations_list, self.n_columns )

        for seq in combinations_list:
            B = np.empty((len(seq),self.X.shape[0]))
            list_of_name = [] #'multiply and devision - '
            for i, tup in enumerate(seq):
                A = np.ones(self.X.shape[0])   
                list_of_name.append(f'multiply on - {" ".join(str(item) for item in tup[:-1])}, and devision on - {tup[-1]}')
                for col in tup:
                    A *= self.X[col].to_numpy()
                    A = A/(self.X[tup[-1]]*self.X[tup[-1]])
                B[i] = A
            yield B, list_of_name


         

    def devision_on_last(self, by=3, column_list=None, with_all_data=True):
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in permutations(name_columns, by):
            combinations_list.append(name_col)
            
        B = np.empty((len(combinations_list),self.X.shape[0]))
        list_of_name = []
        for i, tup in enumerate(combinations_list):
            A = np.ones(self.X.shape[0])
            list_of_name.append(f'multiply on - {" ".join(str(item) for item in tup[:-1])}, and devision on - {tup[-1]}')#.append(f'multiply of - {" ".join(str(item) for item in tup)}')
            for col in tup:
                A *= self.X[col]
                A = A/(self.X[tup[-1]]*self.X[tup[-1]])
                B[i] = A
        df = pd.DataFrame(B.T, columns=list_of_name)
        if with_all_data:
            return pd.concat([df, self.X, self.y], axis=1)   
        else:     
            return df
    
            
    def devision_in_pairs(self, column_list=None, with_all_data=True):
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in permutations(name_columns, 2):
            combinations_list.append(name_col)
        B = np.empty((len(combinations_list),self.X.shape[0]))
        list_of_name = []
        for i, tup in enumerate(combinations_list):
            list_of_name.append(f'{str(tup[0])} // {str(tup[1])}')
            A = X[tup[0]]/X[tup[1]]
            B[i] = A
        df = pd.DataFrame(B.T, columns=list_of_name)
        if with_all_data:
            return pd.concat([df, self.X, self.y], axis=1)   
        else:     
            return df
            


    def log(self, column_list=None):
        name_columns = self.X.columns if column_list == None else column_list
        for col in name_columns:
            self.X['log '+ str(col)] = np.log(self.X[col])    
        df = pd.concat([self.X, self.y], axis=1)  
        return df
        

    def exp(self, column_list=None):
        name_columns = self.X.columns if column_list == None else column_list
        #B = np.empty((len(combinations_list),self.X.shape[0]))
        for col in name_columns:
            self.X['exp '+ str(col)] = np.exp.self.X[col]    
        df = pd.concat([self.X, self.y], axis=1)   
        return df

    def degree(self, by=2, column_list=None, with_all_data=True):
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in permutations(name_columns, 2):
            combinations_list.append(name_col)
        B = np.empty((len(combinations_list),self.X.shape[0]))
        list_of_name = []
        for i, tup in enumerate(combinations_list):
            #print(tup[0])
            list_of_name.append(f'{str(tup[0])} ** {str(tup[1])}')
            A = X[tup[0]]**X[tup[1]]
            B[i] = A
        df = pd.DataFrame(B.T, columns=list_of_name)
        if with_all_data:
            return pd.concat([self.X, self.y], axis=1)   
        else:     
            return df

    def devision_on_last_2(self, by=3, column_list=None,  with_all_data=True):
        combinations_list = []
        name_columns = self.X.columns if column_list == None else column_list
        for name_col in combinations(name_columns, by-1):
            combinations_list.append(name_col)
        A = len(combinations_list)*(self.X.shape[1]-2)

        
        B = np.empty((A, self.X.shape[0]))
        list_of_name = []
        for i, tup in enumerate(combinations_list):
            A = np.ones(self.X.shape[0])
            dev_sub = [el for el in name_columns if el not in tup]

            for col in tup:
                A *= self.X[col]
                for dev in dev_sub:
                    list_of_name.append(f'multiply on - {" ".join(str(item) for item in tup)}, and devision on - {dev}')
                    A = A/self.X[dev]
                B[i] = A
        list_of_name  = list(dict.fromkeys(list_of_name))

        df = pd.DataFrame(B.T, columns=list_of_name)
        if with_all_data:
            return pd.concat([df, self.X, self.y], axis=1)   
        else:     
            return df

    def deltas(self, column_list=None,  with_all_data=True):
        # use with standart scaler or normalize data only
        name_columns = self.X.columns if column_list == None else column_list
        combinations_list = []
        for name_col in combinations(name_columns, 2):
            combinations_list.append(name_col)
        list_of_name = []
        B = np.empty((len(combinations_list),self.X.shape[0]))
        for i, tup in enumerate(combinations_list): 
            list_of_name.append(f'{str(tup[0])} - {str(tup[1])}')
            B[i] = self.X[tup[0]]-self.X[tup[1]]
        df = pd.DataFrame(B.T, columns=list_of_name) #?????
        if with_all_data:
            return pd.concat([df, self.X, self.y], axis=1)   
        else:     
            return df
        
    def sums():
        pass
  
        

class Correlations():
    def __init__(self, X, y):
        self.X = X #2d dimentional df
        self.y = y
    

    def mutual_info(self, mode='regression'):
        mi_dict = {}
        names_col = self.X.columns
        if mode == 'regression':
            v = mutual_info_regression(self.X, self.y)
        elif mode == 'classification':
            v = mutual_info_classif(self.X, self.y)
        else:
            print("mode was no chosen")
        for i, j in zip(names_col, v):
            mi_dict[i]=j
    
        mi_dict = dict(sorted(mi_dict.items(), key=lambda item: item[1], reverse=True))
        mi_df = pd.Series(mi_dict, name='mu_info')
        #    mi_dict = dict(sorted(mi_dict.items(), key=lambda item: item[1], reverse=True))
        return mi_df


    def cramer_with_target(self):
        corr_val_crammer = {}
        for col in self.X.columns:
            confusion_matrix = pd.crosstab(self.X[col], self.y)
            chi2 = stats.chi2_contingency(confusion_matrix)[0]
            n = confusion_matrix.sum().sum()
            r, c = confusion_matrix.shape
            min_dim = min(r - 1, c - 1)
            v = np.sqrt(chi2 / (n * min_dim))
            corr_val_crammer['crammer with target ' + col]= v
            corr_val_df_crammer = pd.Series(corr_val_crammer, name='corr_val_crammer')
        return corr_val_df_crammer


    def correlation(self, name='pearsonr'):
        corr_val = {}
        for col in self.X.columns:
            if name == 'pearsonr':
                res = stats.pearsonr(self.X[col], self.y)
            elif name == 'spearmanr': #Спирмен может быть полезна для нелинейных зависимостей.
                res = stats.spearmanr(self.X[col], self.y)
            elif name == 'kendalltau': #Кендалла, также используемой для ранговых данных
                res = stats.kendalltau(self.X[col], self.y)
            corr_val[col] = res[0]
        corr_val = sorted(corr_val.items(), key=lambda item: item[1], reverse=True)
        corr_val = dict(corr_val)
        corr_val_df = pd.Series(corr_val, name=name)
        return corr_val_df

    def heat_map(self):
        co_mtx = self.X.corr(numeric_only=True)
        w = co_mtx.shape[0]

        figure(figsize=(w, w), dpi=80)
        # Plot correlation heatmap
        sns.heatmap(co_mtx, cmap="YlGnBu", annot=True)



    def corr_feature_analisys(self):
        cor_dict = {}
        names = self.X.columns
        combinations_list = []
        for name_col in combinations(names, 2):
            combinations_list.append(name_col)

        for pairs in combinations_list:
            stat, _ = stats.pearsonr(self.X[pairs[0]], self.X[pairs[1]])
            cor_dict[pairs] = stat
        cor_dict = sorted(cor_dict.items(), key=lambda item: item[1], reverse=True)
        cor_dict = dict(cor_dict)

        return cor_dict

    def vis_corr_feature_analisys(self):
        cor_dict = self.corr_feature_analisys()
        keys = [' '.join(el) for el in cor_dict.keys()]
        vals = [float(v) for v in cor_dict.values()]#[float(cor_dict.values()) for k in keys]
        w = len(keys)
        figure(figsize=(w, w), dpi=80)
        sns.catplot(x=vals, y=keys)

    def full_corr(self, mode='classification'):
        sf1 = self.correlation(name='pearsonr')
        #sf1['spearmanr'] = self.correlation(name='spearmanr')[]
        sf2 = self.correlation(name='spearmanr')
        sf3 = self.correlation(name='kendalltau')
        sf4 = self.mutual_info(mode=mode)
        df1 =  pd.merge(sf1, sf2, how='inner', left_index=True, right_index=True)
        df2 = pd.merge(sf3, sf4, how='inner', left_index=True, right_index=True)
        df3 = pd.merge(df1, df2, how='inner', left_index=True, right_index=True)
        return df3
        
        
  
        

## Corr Analysis

## Multiplay in pairs

In [21]:
df_gen = Generator(heart2[heart2.columns[:-1]], heart2['cardio']).multiply(by=2)
corr = Correlations(df_gen[df_gen.columns[:-1]], df_gen['cardio'])
cor_df =  corr.full_corr(mode='classification')
cor_df

,pearsonr,spearmanr,kendalltau,mu_info
multiply of - age weight,0.270089,0.277935,0.226935,0.039370
multiply of - age cholesterol,0.254517,0.297757,0.243132,0.054637
multiply of - weight cholesterol,0.250368,0.258768,0.212919,0.036105
age,0.238159,0.234429,0.191428,0.033344
multiply of - age height,0.222389,0.219603,0.179307,0.025331
...,...,...,...,...
multiply of - smoke alco,-0.016406,-0.010538,-0.010433,0.005223
multiply of - alco active,-0.030120,-0.034721,-0.034109,0.006330
multiply of - smoke active,-0.035532,-0.037577,-0.036644,0.008135
active,-0.035653,-0.035653,-0.035653,0.005710


## Myltiply in 5

In [22]:
df_gen2_2 = Generator(heart2[heart2.columns[:-1]], heart2['cardio']).multiply(by=5)
corr = Correlations(df_gen2_2[df_gen2_2.columns[:-1]], df_gen2_2['cardio'])
cor_df2 =  corr.full_corr(mode='classification')
cor_df2

,pearsonr,spearmanr,kendalltau,mu_info
multiply of - age height weight cholesterol active,0.242811,0.246223,0.201042,0.035990
multiply of - age height weight cholesterol alco,0.242382,0.290936,0.237550,0.047276
age,0.238159,0.234429,0.191428,0.033929
multiply of - age height weight cholesterol smoke,0.235253,0.278915,0.227735,0.044058
multiply of - age weight cholesterol alco active,0.221943,0.239168,0.195281,0.037115
...,...,...,...,...
alco,-0.007330,-0.007330,-0.007330,0.005993
height,-0.010821,-0.012383,-0.010315,0.000000
smoke,-0.015486,-0.015486,-0.015486,0.006609
multiply of - gender height smoke alco active,-0.020567,-0.020373,-0.016781,0.000544


## Myltiply in 3

In [23]:
df_gen2 = Generator(heart2[heart2.columns[:-1]], heart2['cardio']).multiply(by=3)
corr = Correlations(df_gen2[df_gen2.columns[:-1]], df_gen2_2['cardio'])
cor_df2_2 =  corr.full_corr(mode='classification')
cor_df2_2

,pearsonr,spearmanr,kendalltau,mu_info
multiply of - age weight cholesterol,0.275596,0.319453,0.260835,0.055484
multiply of - age height cholesterol,0.253729,0.290916,0.237534,0.044859
multiply of - age height weight,0.251413,0.258015,0.210670,0.036248
multiply of - height weight cholesterol,0.246332,0.248168,0.202742,0.038194
age,0.238159,0.234429,0.191428,0.031555
...,...,...,...,...
multiply of - gender smoke active,-0.019528,-0.016451,-0.015527,0.000764
multiply of - smoke alco active,-0.030608,-0.033143,-0.032034,0.009537
multiply of - height alco active,-0.031021,-0.033470,-0.027675,0.001581
multiply of - height smoke active,-0.035604,-0.035091,-0.029000,0.000022


## Deltas

In [24]:
df_gen3 = Generator(heart2[heart2.columns[:-1]], heart2['cardio']).deltas()
corr = Correlations(df_gen3[df_gen3.columns[:-1]], df_gen3['cardio'])
cor_df3 =  corr.full_corr(mode='classification')
cor_df3

,pearsonr,spearmanr,kendalltau,mu_info
age - active,0.238165,0.234437,0.191434,0.031560
age - smoke,0.238160,0.234431,0.191429,0.033607
age - alco,0.238159,0.234429,0.191428,0.031622
age,0.238159,0.234429,0.191428,0.033160
age - gender,0.238157,0.234426,0.191426,0.029552
...,...,...,...,...
gender - ap_lo,-0.065701,-0.351237,-0.308611,0.074321
height - ap_lo,-0.066145,-0.283236,-0.234295,0.046793
gender - cholesterol,-0.173536,-0.152034,-0.142788,0.021345
gender - weight,-0.182231,-0.183554,-0.151578,0.017849


## Log

In [47]:
df_gen4 = Generator(heart2[heart2.columns[:-1]], heart2['cardio']).log()
df_gen4 = df_gen4.fillna(0)
df_gen4.replace([np.inf, -np.inf], 10e9, inplace=True)
corr = Correlations(df_gen4[df_gen4.columns[:-1]], df_gen4['cardio'])
cor_df4 = corr.full_corr(mode='classification')
cor_df4

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,pearsonr,spearmanr,kendalltau,mu_info
log ap_hi,0.290687,0.451885,0.401699,0.121887
age,0.238159,0.234429,0.191428,0.032187
log age,0.237071,0.234429,0.191428,0.032623
cholesterol,0.221147,0.215117,0.208824,0.031707
log cholesterol,0.219603,0.215117,0.208824,0.024703
log weight,0.184254,0.182701,0.150995,0.017194
weight,0.181660,0.182701,0.150995,0.018246
log gluc,0.090940,0.091488,0.089827,0.007219
gluc,0.089307,0.091488,0.089827,0.010020
ap_lo,0.065719,0.362572,0.332897,0.072845


In [ ]:
df_gen = Generator(heart2[heart2.columns[:-1]], heart['cardio']).exp()

In [ ]:
df_gen = Generator(heart2[heart2.columns[:-1]], heart['cardio']).degree()

In [ ]:
df_gen = Generator(heart2[heart2.columns[:-1]], heart['cardio']).devision_on_last_2()

'multiply of - ap_hi cholesterol': 0.12868723012547956,
 'ap_hi': 0.12198323105012188,
 'multiply of - ap_hi ap_lo': 0.12012630609887065,
 'multiply of - ap_hi gluc': 0.11764224083125385,
 'multiply of - height ap_hi': 0.11645413883453792,
 'multiply of - gender ap_hi': 0.11547996142741934,
 'multiply of - age ap_hi': 0.11065677496669246,
 'multiply of - ap_hi active': 0.10546168339602358,
 'multiply of - weight ap_hi': 0.10001372310340795,
 'multiply of - ap_lo cholesterol': 0.09105549758079867,
 'multiply of - age ap_lo': 0.08670456372350466,
 'ap_lo': 0.07551061014232241,
 'multiply of - gender ap_lo': 0.07538714887498865,
 'multiply of - ap_lo gluc': 0.07376476593987347,
 'multiply of - weight ap_lo': 0.0719285541041863,
 'multiply of - height ap_lo': 0.06763992499346316



 'multiply of - gender ap_hi cholesterol': 0.12642497401288,
 'multiply of - ap_hi ap_lo cholesterol': 0.12545853324937428,
 'multiply of - ap_hi cholesterol gluc': 0.12432499530081875,
 'multiply of - height ap_hi cholesterol': 0.12229630186528717,
 'ap_hi': 0.12140187751284182,
 'multiply of - age ap_hi ap_lo': 0.11790270802617608,
 'multiply of - gender ap_hi gluc': 0.11589606815414855,
 'multiply of - ap_hi ap_lo gluc': 0.1156998028947478,
 'multiply of - height ap_hi gluc': 0.11425469929383314,
 'multiply of - gender height ap_hi': 0.11269482570743627,
 'multiply of - ap_hi cholesterol active': 0.11102680594378822,
 'multiply of - gender ap_hi ap_lo': 0.11101527069697248,
 'multiply of - height ap_hi ap_lo': 0.11096273853804406,
 'multiply of - age ap_hi cholesterol': 0.11042740001336715,
 'multiply of - weight ap_hi cholesterol': 0.1094161010929422,
 'multiply of - weight ap_hi ap_lo': 0.10857167600496864,
 'multiply of - ap_hi gluc active': 0.1030877749085315,
 'multiply of - age ap_hi gluc': 0.10066613393303792,
 'multiply of - age height ap_hi': 0.09914599666884416,
 'multiply of - ap_hi ap_lo active': 0.0987815103799059,
 'multiply of - height ap_hi active': 0.09717783472266173,
 'multiply of - gender ap_hi active': 0.09700145061052501

 

In [4]:
def train_test_scaller(df, name=''):
    
    y = df[name]
    X = df.drop([name], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=99)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

def auto_trainer(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(n_estimators=13, max_depth=5, random_state=42).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    #y_pred_proba = clf.predict_proba(X_test[:2, :])
    clf.score(X_test, y_test)
   
    print('Report on test data')
    print(classification_report(y_test, y_pred))
    cf_matrix = confusion_matrix(y_test, y_pred)

    print('Report on train data')
    print(classification_report(y_train, y_pred_train))
    cf_matrix = confusion_matrix(y_train, y_pred_train)
    print(cf_matrix)

def feature_gen(df_gen, target_name, threshold=0.1): 
   ## add some chose
    corr = Correlations(df_gen[df_gen.columns[:-1]], df_gen[target_name])
    cor_dict = corr.corr_feature_analisys()
    mi = corr.mutual_info()
    mi_lim = {k:v for k, v in mi.items() if v < threshold} # exclude columns with have a mu info with target less than 0.1
    mi_lim_ex = [col for col in mi_lim.keys() if col in df_gen.columns]
    df_gen = df_gen.drop(columns=mi_lim_ex, axis=1)
    return df_gen

def simple_pipe(df_inner, df_gen, target_name, threshold):
    ndf = feature_gen(df_gen, target_name, threshold)
    X_train, X_test, y_train, y_test = train_test_scaller(df_inner, name=target_name)
    print('W/O manipulations')
    auto_trainer(X_train, X_test, y_train, y_test)
    print('With manipulations')
    X_train, X_test, y_train, y_test = train_test_scaller(ndf, name=target_name)
    auto_trainer(X_train, X_test, y_train, y_test)


In [5]:
df_gen = Generator(heart[heart.columns[:-1]], heart['DEATH_EVENT']).multiply(by=2)
simple_pipe(heart, df_gen, 'DEATH_EVENT', 0.1)

W/O manipulations
Report on test data
              precision    recall  f1-score   support

           0       0.88      0.88      0.88        52
           1       0.74      0.74      0.74        23

    accuracy                           0.84        75
   macro avg       0.81      0.81      0.81        75
weighted avg       0.84      0.84      0.84        75

Report on train data
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       151
           1       0.97      0.85      0.91        73

    accuracy                           0.94       224
   macro avg       0.95      0.92      0.93       224
weighted avg       0.94      0.94      0.94       224

[[149   2]
 [ 11  62]]
With manipulations
Report on test data
              precision    recall  f1-score   support

           0       0.88      0.96      0.92        52
           1       0.89      0.70      0.78        23

    accuracy                           0.88        75
   macr

In [16]:
df_gen = Generator(person[person.columns[:-1]], person['Personality']).multiply(by=2)
simple_pipe(person, df_gen, 'Personality', 0.1)

W/O manipulations
Report on test data
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       342
           1       0.93      0.92      0.93       383

    accuracy                           0.92       725
   macro avg       0.92      0.92      0.92       725
weighted avg       0.92      0.92      0.92       725

Report on train data
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1067
           1       0.95      0.93      0.94      1108

    accuracy                           0.94      2175
   macro avg       0.94      0.94      0.94      2175
weighted avg       0.94      0.94      0.94      2175

[[1015   52]
 [  82 1026]]
With manipulations
Report on test data
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       342
           1       0.93      0.92      0.93       383

    accuracy                           0.92       725
   

In [21]:
df_gen = Generator(iris[iris.columns[:-1]], iris['Species']).multiply(by=2)
simple_pipe(iris, df_gen, 'Species', 0.1)

W/O manipulations
Report on test data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.93      0.93      0.93        14
           2       0.92      0.92      0.92        12

    accuracy                           0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

Report on train data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       1.00      1.00      1.00        36
           2       1.00      1.00      1.00        38

    accuracy                           1.00       112
   macro avg       1.00      1.00      1.00       112
weighted avg       1.00      1.00      1.00       112

[[38  0  0]
 [ 0 36  0]
 [ 0  0 38]]
With manipulations
Report on test data
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     

In [55]:
df_gen = Generator(marketing[marketing.columns[:-1]], marketing['responded']).multiply(by=2)
simple_pipe(marketing, df_gen, 'responded', 0.1)

W/O manipulations
Report on test data
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         7
         2.0       1.00      1.00      1.00         7

    accuracy                           1.00        14
   macro avg       1.00      1.00      1.00        14
weighted avg       1.00      1.00      1.00        14

Report on train data
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00        21
         2.0       1.00      1.00      1.00        21

    accuracy                           1.00        42
   macro avg       1.00      1.00      1.00        42
weighted avg       1.00      1.00      1.00        42

[[21  0]
 [ 0 21]]
With manipulations
Report on test data
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         7
         2.0       1.00      1.00      1.00         7

    accuracy                           1.00        14
   macro av

In [61]:
df_gen = Generator(bankruptcy[bankruptcy.columns[:-1]], bankruptcy['Bankrupt?']).multiply(by=2)
simple_pipe(bankruptcy, df_gen, 'Bankrupt?', 0.1)

/tmp/ipykernel_38/2262713051.py:262: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  stat, _ = stats.pearsonr(self.X[pairs[0]], self.X[pairs[1]])
/usr/local/lib/python3.11/dist-packages/scipy/stats/_stats_py.py:4638: RuntimeWarning: invalid value encountered in less
  nconst_y = xp.any(normym < threshold*xp.abs(ymean), axis=axis)
/usr/local/lib/python3.11/dist-packages/scipy/_lib/array_api_compat/common/_aliases.py:354: RuntimeWarning: invalid value encountered in less
  ia = (out < a) | xp.isnan(a)
/usr/local/lib/python3.11/dist-packages/scipy/_lib/array_api_compat/common/_aliases.py:361: RuntimeWarning: invalid value encountered in greater
  ib = (out > b) | xp.isnan(b)


KeyboardInterrupt: 

In [79]:
df_gen = Generator(heart2[heart2.columns[:-1]], heart2['cardio']).multiply(by=2)
simple_pipe(heart2, df_gen, 'cardio', 0.1)

W/O manipulations
Report on test data
              precision    recall  f1-score   support

           0       0.71      0.79      0.75      8705
           1       0.77      0.67      0.72      8795

    accuracy                           0.73     17500
   macro avg       0.74      0.73      0.73     17500
weighted avg       0.74      0.73      0.73     17500

Report on train data
              precision    recall  f1-score   support

           0       0.70      0.80      0.75     26316
           1       0.76      0.66      0.71     26184

    accuracy                           0.73     52500
   macro avg       0.73      0.73      0.73     52500
weighted avg       0.73      0.73      0.73     52500

[[20924  5392]
 [ 8844 17340]]
With manipulations
Report on test data
              precision    recall  f1-score   support

           0       0.71      0.79      0.75      8705
           1       0.77      0.68      0.72      8795

    accuracy                           0.73     17500

In [ ]:
df_gen = Generator(onlinefraud[onlinefraud.columns[:-1]], onlinefraud['isFraud']).multiply(by=2)
simple_pipe(onlinefraud, df_gen, 'isFraud', 0.1)

In [ ]:
df_gen = Generator(creditfraud[onlinefraud.columns[:-1]], onlinefraud['Class']).multiply(by=2)
simple_pipe(onlinefraud, df_gen, 'Class', 0.1)

In [ ]:
df_gen = Generator(bankruptcy[bankruptcy.columns[:-1]], bankruptcy['Bankrupt?']).multiply(by=2)
simple_pipe(bankruptcy, df_gen, 'Bankrupt?', 0.1)

In [ ]:
def train_test_scaller(df, name=''):
    
    y = df[name]
    X = df.drop([name], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=99)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

def auto_trainer(X_train, X_test, y_train, y_test):
    clf = RandomForestRegressor(n_estimators=13, max_depth=5, random_state=42).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_pred_train = clf.predict(X_train)
    #y_pred_proba = clf.predict_proba(X_test[:2, :])
    clf.score(X_test, y_test)
   
    print('Report on test data')
    print(classification_report(y_test, y_pred))
    cf_matrix = confusion_matrix(y_test, y_pred)
    #plt.figure()
    #sns.heatmap(cf_matrix, annot=True)
    #$print(cf_matrix)
    
    print('Report on train data')
    print(classification_report(y_train, y_pred_train))
    cf_matrix = confusion_matrix(y_train, y_pred_train)
    print(cf_matrix)
    #plt.figure()
    #sns.heatmap(cf_matrix, annot=True)


#def feature_gen(df_inner, target_name, threshold=0.1): 
#    df = Generator(df_inner[df_inner.columns[:-1]], df_inner[target_name]).multiply(by=2) ## add some chose
#    corr = Correlations(df[df.columns[:-1]], df[target_name])
#    cor_dict = corr.corr_feature_analisys()
#    mi = corr.mutual_info()
#    mi_lim = {k:v for k, v in mi.items() if v < threshold} # exclude columns with have a mu info with target less than 0.1
#    mi_lim_ex = [col for col in mi_lim.keys() if col in df.columns]
#    df = df.drop(columns=mi_lim_ex, axis=1)
#    return df

#df_gen = Generator(heart[heart.columns[:-1]], heart['DEATH_EVENT']).multiply(by=2)
def feature_gen(df_gen, target_name, threshold=0.1): 
   ## add some chose
    corr = Correlations(df_gen[df_gen.columns[:-1]], df_gen[target_name])
    cor_dict = corr.corr_feature_analisys()
    mi = corr.mutual_info()
    mi_lim = {k:v for k, v in mi.items() if v < threshold} # exclude columns with have a mu info with target less than 0.1
    mi_lim_ex = [col for col in mi_lim.keys() if col in df_gen.columns]
    df_gen = df_gen.drop(columns=mi_lim_ex, axis=1)
    return df_gen

def simple_pipe(df_inner, df_gen, target_name, threshold):
    ndf = feature_gen(df_gen, target_name, threshold)
    X_train, X_test, y_train, y_test = train_test_scaller(df_inner, name=target_name)
    print('W/O manipulations')
    auto_trainer(X_train, X_test, y_train, y_test)
    print('With manipulations')
    X_train, X_test, y_train, y_test = train_test_scaller(ndf, name=target_name)
    auto_trainer(X_train, X_test, y_train, y_test)


In [42]:
df_gen = Generator(student[student.columns[:-1]], student['exam_score']).multiply(by=2)
simple_pipe(student, df_gen, 'exam_score', 0.1)

/usr/local/lib/python3.11/dist-packages/scipy/stats/_stats_py.py:4637: RuntimeWarning: invalid value encountered in less
  nconst_x = xp.any(normxm < threshold*xp.abs(xmean), axis=axis)
/usr/local/lib/python3.11/dist-packages/scipy/stats/_stats_py.py:4638: RuntimeWarning: invalid value encountered in less
  nconst_y = xp.any(normym < threshold*xp.abs(ymean), axis=axis)
/usr/local/lib/python3.11/dist-packages/scipy/_lib/array_api_compat/common/_aliases.py:354: RuntimeWarning: invalid value encountered in less
  ia = (out < a) | xp.isnan(a)
/usr/local/lib/python3.11/dist-packages/scipy/_lib/array_api_compat/common/_aliases.py:361: RuntimeWarning: invalid value encountered in greater
  ib = (out > b) | xp.isnan(b)


ValueError: Input X contains NaN.

In [37]:
student['gender'].unique()

array([1, 2, 'Other'], dtype=object)

In [ ]:
df = feature_gen(iris, 'Personality')
X_train, X_test, y_train, y_test = train_test_scaller(df, name='Personality')
auto_trainer(X_train, X_test, y_train, y_test )